# Parse Charged Molecules
We also have the G4MP2 energies of these molecules. This script reads them in from the format I was sent, and puts them into a DataFrame.

In [1]:
from jcesr_ml.utils import compute_atomization_energy
from dlhub_sdk.models.datasets import TabularDataset
from pymatgen.io.xyz import XYZ
from ase.io.xyz import read_xyz
import pybel
from pybel import readstring
from io import StringIO
from glob import glob
from ase.units import Hartree, eV
from tqdm import tqdm
import pandas as pd
import json
import os

In [2]:
output_path = os.path.join('data', 'output', 'ip_molecules_3405_paper.json')

## Load in B3LYP & wB97Xd Data
This was computed separately, and is available in a separate cvs file

In [3]:
charged_data = pd.read_csv(os.path.join('data', 'input', 'GDB9', 'ip_molecules_3405_paper.csv' ), 
                           delim_whitespace='')

In [4]:
charged_data.head()

,filename,n_heavy_atoms,n_atoms,n_electrons,xyz,xyz_oxidized,u0_b3lyp,u0_g4mp2,zpe_b3lyp,electronic_oxidized_b3lyp,...,electronic_neutral_b3lyp,electronic_neutral_wB97Xd,electronic_neutral_b3lyp_large,u0_oxidized_b3lyp,u0_oxidized_g4mp2,zpe_oxidized_b3lyp,u0_b3lyp_large,u0_wB97Xd,u0_oxidized_b3lyp_large,u0_oxidized_wB97Xd
0,dsgdb9nsd_000001,1,5,10,5\nH4 C1\nC -0.012698 1.085804 0.008001\nH 0.0...,5\n\nC -3.6e-05 0.000157 4.4e-05\nH -1.031842 ...,-40.478930,-40.427662,0.044749,-40.060622,...,-40.523679,-40.518353,-40.526016,-40.024627,-39.959837,0.035995,-40.481268,-40.473604,-40.024679,-40.016973
1,dsgdb9nsd_000002,1,4,10,4\nH3 N1\nN -0.040426 1.024108 0.062564\nH 0.0...,4\n\nN 0.0 4.8e-05 0.000313\nH -0.889576 0.513...,-56.525887,-56.478971,0.034358,-56.194430,...,-56.560245,-56.564755,-56.568919,-56.161757,-56.104750,0.032673,-56.534561,-56.530397,-56.161908,-56.158444
2,dsgdb9nsd_000003,1,3,10,3\nH2 O1\nO -0.034360 0.977540 0.007602\nH 0.0...,3\n\nO -0.0 0.115838 -0.0\nH 0.824472 -0.46335...,-76.404702,-76.355852,0.021375,-75.975303,...,-76.426077,-76.437293,-76.440542,-75.956721,-75.893134,0.018582,-76.419167,-76.415918,-75.957305,-75.954312
3,dsgdb9nsd_000005,2,3,14,3\nH1 C1 N1\nC -0.013324 1.132466 0.008276\nN ...,3\n\nC 0.019872 -0.52686 0.0\nN 0.019872 0.672...,-93.411888,-93.312546,0.016601,-92.938513,...,-93.428489,-93.420228,-93.434353,-92.925709,-92.814499,0.012804,-93.417752,-93.403627,-92.924130,-92.910367
4,dsgdb9nsd_000006,2,4,16,4\nH2 C1 O1\nC -0.013978 1.180211 0.007752\nO ...,4\n\nC 0.0 -0.525843 -0.0\nO 0.0 0.66622 0.0\n...,-114.483613,-114.371590,0.026603,-114.118908,...,-114.510216,-114.506165,-114.519856,-114.094926,-113.971387,0.023982,-114.493253,-114.479562,-114.096882,-114.084381


Sort it into a reproducible order

In [5]:
def get_counts(xyz):
    """Given the XYZ coordinates of a molecule, compute its size
    
    Args:
        xyz (string): XYZ-format file of the atomic structure
    Returns:
        (dict) Different measures of molecular size:
            n_heavy_atom (int): Number of heavy (i.e., non-Hydrogen) atoms
            n_electrons (int): Number of electrons in the system
    """
    
    mol = read_xyz(StringIO(xyz)).__next__()
    Z = mol.get_atomic_numbers()
    return {
        'n_atoms': len(Z),
        'n_heavy_atoms': (Z > 1).sum(),
        'n_electrons': Z.sum(), 
    }

In [6]:
def func(xyz):
    """Given the XYZ coordinates of a molecule, compute its size
    
    Args:
        xyz (string): XYZ-format file of the atomic structure
    Returns:
        (dict) Different measures of molecular size:
            n_heavy_atom (int): Number of heavy (i.e., non-Hydrogen) atoms
            n_electrons (int): Number of electrons in the system
    """
    
    mol = pybel.readstring('xyz', xyz)
    return mol.write('smi').split("\t")[0]


In [7]:
charged_data['smiles'] = charged_data['xyz'].apply(func)

In [8]:
charged_data['b3lyp_asc_neutral'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_b3lyp'], 'b3lypa_ip_neutral'),
    axis=1
)

charged_data['b3lyp_asc_cation'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_oxidized_b3lyp'], 'b3lypa_ip_cation'),
    axis=1
)

In [9]:
charged_data['b3lyp_large_asc_neutral'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_b3lyp_large'], 'b3lypa_large_ip_neutral'),
    axis=1
)

charged_data['b3lyp_large_asc_cation'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_oxidized_b3lyp_large'], 'b3lypa_large_ip_cation'),
    axis=1
)

In [10]:
charged_data['wB97Xd_asc_neutral'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_wB97Xd'], 'wB97Xda_ip_neutral'),
    axis=1
)

charged_data['wB97Xd_asc_cation'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_oxidized_wB97Xd'], 'wB97Xda_ip_cation'),
    axis=1
)

In [11]:
charged_data['g4mp2_asc_neutral'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_g4mp2'], 'g4mp2a_ip_neutral'),
    axis=1
)

charged_data['g4mp2_asc_cation'] = charged_data.apply(
    lambda x: compute_atomization_energy(next(read_xyz(StringIO(x['xyz']))), x['u0_oxidized_g4mp2'], 'g4mp2a_ip_cation'),
    axis=1
)

In [12]:
charged_data['source'] = 'redox_molecules'

In [13]:
charged_data['G4MP2_IP'] = (charged_data['u0_oxidized_g4mp2'] 
                            - charged_data['u0_g4mp2']) * Hartree / eV

charged_data['G4MP2_IP_ASC'] = (charged_data['g4mp2_asc_cation'] 
                                - charged_data['g4mp2_asc_neutral']) * Hartree / eV


charged_data['wB97Xd_IP'] = (charged_data['u0_oxidized_wB97Xd'] 
                                 - charged_data['u0_wB97Xd']) * Hartree / eV

charged_data['wB97Xd_IP_ASC'] = (charged_data['wB97Xd_asc_cation'] 
                                      - charged_data['wB97Xd_asc_neutral']) * Hartree / eV


charged_data['B3LYP_IP'] = (charged_data['u0_oxidized_b3lyp'] 
                                - charged_data['u0_b3lyp']) * Hartree / eV

charged_data['B3LYP_IP_ASC'] = (charged_data['b3lyp_asc_cation'] 
                                    - charged_data['b3lyp_asc_neutral']) * Hartree / eV


charged_data['B3LYP_large_IP'] = (charged_data['u0_oxidized_b3lyp_large'] 
                                - charged_data['u0_b3lyp_large']) * Hartree / eV

charged_data['B3LYP_large_IP_ASC'] = (charged_data['b3lyp_large_asc_cation'] 
                                    - charged_data['b3lyp_large_asc_neutral']) * Hartree / eV




Sort it into a reproducible order

In [14]:
charged_data.sort_values('filename', inplace=True)

## Mark a Test Set
Just in case we want to do any kind of hold-out test later

In [15]:
charged_data['in_holdout'] = False

In [16]:
charged_data.loc[charged_data.sample(frac=0.1, random_state=1).index, 'in_holdout'] = True

## Save the Data with a Description
Use the DLHub SDK to describe this data

In [17]:
charged_data.to_json(output_path, 'records', lines=True)

Make the description

In [18]:
metadata = TabularDataset.create_model(output_path, format='json', read_kwargs={'lines': True})

In [19]:
metadata.set_title('IPs of {} GDB9_heavyatoms_charged Molecules'.format(len(charged_data)))
metadata.set_name('ip_molecules_3405_paper')

TBD: Get other authors from Rajeev

In [20]:
metadata.set_authors(['Dandu, Naveen', 'Ward, Logan', 'Assary, Rajeev', 'Narayanan, Badri',  'Curtiss, Larry'],
                     [['Argonne National Laboratory']]*5)

In [21]:
metadata.annotate_column('filename', 'Filename, which contains the name of the molecule')
metadata.annotate_column('source', 'Source of the dataset')

In [22]:
metadata.annotate_column('in_holdout', 'Whether the entry is in the test set')

In [23]:
metadata.annotate_column('n_atoms', 'Number of atoms')
metadata.annotate_column('n_electrons', 'Number of electrons')
metadata.annotate_column('n_heavy_atoms', 'Number of non-hydrogen atoms')


In [24]:
#metadata.annotate_column('smiles_neutral', 'SMILES string after relaxation', data_type='string')
metadata.annotate_column('xyz', 'XYZ coordinates after relaxation', data_type='XYZ file')
#metadata.annotate_column('smiles_ox', 'SMILES string after relaxation', data_type='string')
#metadata.annotate_column('xyz_reduced', 'XYZ coordinates after reduced', data_type='XYZ file')


In [25]:
metadata.mark_inputs([ 'xyz' ])

In [26]:
#assert len(metadata.get_unannotated_columns()) == 0

In [27]:
with open(os.path.join(os.path.dirname(output_path),
                       '{}_description.json'.format(metadata['dlhub']['name'])), 'w') as fp:
    json.dump(metadata.to_dict(), fp, indent=2)

In [28]:
charged_data.to_csv('ip_molecules_3405_paper.csv', index=False)